In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from sklearn.linear_model import LogisticRegression
from itertools import combinations
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
arena_data = load_dataset("lmarena-ai/arena-human-preference-140k")['train']
comparisons = pd.read_csv("./data/pairwise_results.csv")
arena_df = arena_data.to_pandas()

README.md: 0.00B [00:00, ?B/s]

In [3]:
arena_df.head()

,id,model_a,model_b,winner,evaluation_session_id,evaluation_order,conversation_a,conversation_b,full_conversation,conv_metadata,category_tag,language,is_code,timestamp
0,c4b9710c-8d64-4bee-a0b0-94637ae4cc65,gemini-2.5-pro,claude-3-7-sonnet-20250219-thinking-32k,model_a,a333a685-37f9-474d-b703-f079d8329552,1,"[{'role': 'user', 'content': [{'type': 'text',...","[{'role': 'user', 'content': [{'type': 'text',...","[{'user': {'role': 'user', 'content': [{'type'...","{'sum_assistant_a_tokens': 1854, 'header_count...",{'creative_writing_v0.1': {'creative_writing':...,en,False,2025-05-29 14:48:41.602925
1,7c44a466-fd07-4992-9764-176b75746a07,claude-3-5-haiku-20241022,claude-3-5-sonnet-20241022,tie,73e84b7b-d158-4d2d-9a7f-56286ec14564,1,"[{'role': 'user', 'content': [{'type': 'text',...","[{'role': 'user', 'content': [{'type': 'text',...","[{'user': {'role': 'user', 'content': [{'type'...","{'sum_assistant_a_tokens': 605, 'header_count_...",{'creative_writing_v0.1': {'creative_writing':...,pl,True,2025-07-07 02:04:51.977824
2,46e04015-dc69-442a-8b37-3caea0336541,o3-mini,gemma-3n-e4b-it,both_bad,8132faad-e9c2-48ef-b608-718504d9ab75,1,"[{'role': 'user', 'content': [{'type': 'text',...","[{'role': 'user', 'content': [{'type': 'text',...","[{'user': {'role': 'user', 'content': [{'type'...","{'sum_assistant_a_tokens': 2216, 'header_count...",{'creative_writing_v0.1': {'creative_writing':...,en,True,2025-07-12 21:43:23.646116
3,4a4380bb-bbdb-495f-8e09-39a08d88a28f,claude-sonnet-4-20250514-thinking-32k,gemini-2.5-pro,model_a,d1c66159-2b6f-4c09-a090-165520330cc3,3,"[{'role': 'user', 'content': [{'type': 'text',...","[{'role': 'user', 'content': [{'type': 'text',...","[{'user': {'role': 'user', 'content': [{'type'...","{'sum_assistant_a_tokens': 297, 'header_count_...",{'creative_writing_v0.1': {'creative_writing':...,en,False,2025-07-09 19:05:16.179693
4,ce23b477-d72c-4048-a01d-c3d26c80ad71,claude-3-5-sonnet-20241022,mistral-medium-2505,model_b,8c2ce87c-650c-4777-8503-65b5f797ec50,1,"[{'role': 'user', 'content': [{'type': 'text',...","[{'role': 'user', 'content': [{'type': 'text',...","[{'user': {'role': 'user', 'content': [{'type'...","{'sum_assistant_a_tokens': 459, 'header_count_...",{'creative_writing_v0.1': {'creative_writing':...,de,False,2025-05-22 10:34:25.921196


In [4]:
arena_df['model_a'].unique()

array(['gemini-2.5-pro', 'claude-3-5-haiku-20241022', 'o3-mini',
       'claude-sonnet-4-20250514-thinking-32k',
       'claude-3-5-sonnet-20241022', 'grok-3-preview-02-24',
       'qwen3-235b-a22b', 'llama-4-scout-17b-16e-instruct',
       'gemini-2.5-flash-lite-preview-06-17-thinking',
       'chatgpt-4o-latest-20250326', 'qwen3-30b-a3b',
       'mistral-medium-2505', 'gpt-4.1-mini-2025-04-14',
       'magistral-medium-2506', 'gemini-2.5-flash-preview-04-17',
       'mistral-small-2506', 'claude-sonnet-4-20250514',
       'amazon-nova-experimental-chat-05-14', 'command-a-03-2025',
       'qwen3-235b-a22b-no-thinking', 'o4-mini-2025-04-16',
       'gpt-4.1-2025-04-14', 'gemma-3-27b-it', 'claude-opus-4-20250514',
       'deepseek-v3-0324', 'deepseek-r1-0528',
       'claude-3-7-sonnet-20250219',
       'llama-4-maverick-03-26-experimental', 'o3-2025-04-16',
       'grok-3-mini-beta', 'gemini-2.0-flash-001', 'qwq-32b',
       'gemini-2.5-flash', 'qwen-max-2025-01-25',
       'llama-3.3-